In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [8]:

class FederatedClient:
    """
    Classe représentant un client (assureur) dans le système d'apprentissage fédéré
    """
    def __init__(self, name, data, features, target):
        self.name = name
        self.data = data
        self.features = features
        self.target = target
        self.model = None
        self.weights = None
        self.prepare_data()
    
    def prepare_data(self):
        """
        Prépare les données pour l'entraînement
        """
        
        if "Exposure" not in self.data.columns:
            raise ValueError(f"'Exposure' requise pour {self.name}")
        
        X = self.data[self.features]
        y = self.data[self.target]
        exposure = self.data["Exposure"]
        
        self.X_train, self.X_test, self.y_train, self.y_test, self.exposure_train, self.exposure_test = train_test_split(
            X, y, exposure, test_size=0.2, random_state=42
        )
    
    def train_local_model(self):
        """
        Entraîne un modèle GLM (régression logistique) localement
        en utilisant les poids précédemment assignés comme point de départ
        """
        if self.model is None:
            self.model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
        
        # Si warm_start est True, l'entraînement utilisera les poids déjà assignés
        # comme point de départ si ils existent
        self.model.warm_start = True
        self.model.fit(self.X_train, self.y_train, sample_weight=self.exposure_train)
        self.weights = np.concatenate([self.model.intercept_, self.model.coef_[0]])
        
        print(f"\nPoids du modèle local pour {self.name}:")
        feature_names = ['Intercept'] + self.features
        for i, (name, weight) in enumerate(zip(feature_names, self.weights)):
            print(f"  {name}: {weight:.6f}")
        
        self.evaluate_model()
        
        return self.weights
    
    def update_weights(self, global_weights):
        """
        Met à jour les poids du modèle local avec les poids globaux
        """
        self.weights = global_weights.copy()  # Utiliser copy() pour éviter les références
        
        # Si le modèle existe déjà, mettre à jour ses coefficients
        if self.model is not None:
            self.model.intercept_ = np.array([global_weights[0]])
            self.model.coef_ = global_weights[1:].reshape(1, -1)
    
    def evaluate_model(self):
        """
        Évalue le modèle sur les données de test
        """
        if self.model is None:
            print(f"Aucun modèle entraîné pour {self.name}")
            return None
        
        # Utiliser predict_proba pour observer les probabilités
        y_proba = self.model.predict_proba(self.X_test)[:, 1]
        
        # Utiliser un seuil plus adapté aux données déséquilibrées
        # (on peut ajuster ce seuil selon la distribution des données)
        threshold = 0.3
        y_pred = (y_proba >= threshold).astype(int)
        
        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred, zero_division=0)
        recall = recall_score(self.y_test, y_pred, zero_division=0)
        f1 = f1_score(self.y_test, y_pred, zero_division=0)
        
        print(f"Performance du modèle pour {self.name}:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1 Score: {f1:.4f}")
        
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }


In [9]:

class FederatedServer:
    """
    Classe représentant le serveur central dans le système d'apprentissage fédéré
    """
    def __init__(self):
        self.clients = []
        self.global_weights = None
    
    def add_client(self, client):
        """
        Ajoute un client au système
        """
        self.clients.append(client)
    
    def federated_averaging(self, weights_list, sample_sizes=None):
        """
        Implémente l'algorithme FedAvg pour agréger les poids
        
        min F(w) := ∑ pk Fk(w) où pk sont les poids d'importance de chaque client
        """
        if sample_sizes is None:
            # Si aucune taille d'échantillon n'est fournie, utiliser une pondération uniforme
            weights = np.mean(weights_list, axis=0)
        else:
            total_samples = sum(sample_sizes)
            weights_array = np.array(weights_list)
            weights = np.zeros_like(weights_array[0])
            
            for i, client_weights in enumerate(weights_array):
                pk = sample_sizes[i] / total_samples  # Poids d'importance
                weights += pk * client_weights
        
        return weights
    
    def train_global_model(self, num_rounds=3):
        """
        Entraîne le modèle global en utilisant l'apprentissage fédéré
        """
        print(f"\nDémarrage de l'apprentissage fédéré avec {len(self.clients)} clients")
        
        # Initialisation des poids globaux (w0)
        if self.global_weights is None and len(self.clients) > 0:
            first_client = self.clients[0]
            num_features = len(first_client.features)
            self.global_weights = np.zeros(num_features + 1)  # +1 pour l'intercept
        
        for round_num in range(num_rounds):
            print(f"\n--- Cycle d'apprentissage fédéré {round_num + 1}/{num_rounds} ---")
            
            # D'abord mettre à jour les poids des clients avec les poids globaux précédents
            if round_num > 0:
                for client in self.clients:
                    client.update_weights(self.global_weights)
            
            local_weights = []
            sample_sizes = []
            
            for client in self.clients:
                # Entraînement du modèle local à partir des poids globaux
                weights = client.train_local_model()
                local_weights.append(weights)
                # Utiliser la taille du training set au lieu du dataset complet
                sample_sizes.append(len(client.X_train))
            
            # Agréger les poids avec l'algorithme FedAvg
            self.global_weights = self.federated_averaging(local_weights, sample_sizes)
            
            print(f"\nPoids globaux après le cycle {round_num + 1}:")
            if len(self.clients) > 0:
                feature_names = ['Intercept'] + self.clients[0].features
                for i, (name, weight) in enumerate(zip(feature_names, self.global_weights)):
                    print(f"  {name}: {weight:.6f}")
        
        return self.global_weights



In [10]:

class FederatedLearning:
    """
    Classe principale pour coordonner l'apprentissage fédéré
    """
    def __init__(self, data_dict, features, target):
        """
        Initialise le système d'apprentissage fédéré
        
        Args:
            data_dict: Dictionnaire {nom_client: dataframe_pandas}
            features: Liste des variables explicatives
            target: Nom de la variable cible
        """
        self.data_dict = data_dict
        self.features = features
        self.target = target
        self.server = FederatedServer()
    
    def setup(self):
        """
        Configure le système en créant les clients
        """
        for client_name, data in self.data_dict.items():
            try:
                print(f"Configuration du client '{client_name}' avec {len(data)} observations")
                
                # Création du client
                client = FederatedClient(client_name, data, self.features, self.target)
                self.server.add_client(client)
                
                print(f"Client '{client_name}' ajouté avec succès")
            except Exception as e:
                print(f"Erreur lors de la configuration du client '{client_name}': {str(e)}")
    
    def train(self, num_rounds=5):  # Augmenter le nombre de cycles par défaut
        """
        Lance l'entraînement fédéré
        
        Args:
            num_rounds: Nombre de cycles d'entraînement
            
        Returns:
            Les poids du modèle global final
        """
        return self.server.train_global_model(num_rounds)



In [11]:

def main():
    """
    Fonction principale
    """
    try:
        features = ['DriverAge', 'Gender', 'VehiculeUsage']
        target = 'Sinistre'
        
        fr = pd.read_csv('french_data.csv')
        be = pd.read_csv('belgium_data.csv')
        eu = pd.read_csv('european_data.csv')
        
        # Normalisation des variables numériques pour faciliter la convergence
        for df in [fr, be, eu]:
            df['DriverAge'] = (df['DriverAge'] - df['DriverAge'].mean()) / df['DriverAge'].std()
        
        data_dict = {
            'France': fr,
            'Belgium': be,
            'Europe': eu
        }
        
        print("Configuration du système d'apprentissage fédéré...")
        fl_system = FederatedLearning(data_dict, features, target)
        fl_system.setup()
        
        print("\nDémarrage de l'entraînement...")
        global_weights = fl_system.train(num_rounds=5)  # Augmenter le nombre de cycles
        
        print("\nApprentissage fédéré terminé!")
        print("Poids finaux du modèle global:")
        feature_names = ['Intercept'] + features
        for i, (name, weight) in enumerate(zip(feature_names, global_weights)):
            print(f"  {name}: {weight:.6f}")
    
    except Exception as e:
        print(f"Erreur lors de l'exécution: {str(e)}")


if __name__ == "__main__":
    main()


Configuration du système d'apprentissage fédéré...
Configuration du client 'France' avec 243065 observations
Client 'France' ajouté avec succès
Configuration du client 'Belgium' avec 163212 observations
Client 'Belgium' ajouté avec succès
Configuration du client 'Europe' avec 237319 observations
Client 'Europe' ajouté avec succès

Démarrage de l'entraînement...

Démarrage de l'apprentissage fédéré avec 3 clients

--- Cycle d'apprentissage fédéré 1/5 ---

Poids du modèle local pour France:
  Intercept: 0.496634
  DriverAge: -0.062988
  Gender: -0.003088
  VehiculeUsage: -0.206355
Performance du modèle pour France:
  Accuracy: 0.0649
  Precision: 0.0649
  Recall: 1.0000
  F1 Score: 0.1218

Poids du modèle local pour Belgium:
  Intercept: 0.060798
  DriverAge: -0.214440
  Gender: 0.029563
  VehiculeUsage: -0.033501
Performance du modèle pour Belgium:
  Accuracy: 0.1121
  Precision: 0.1121
  Recall: 1.0000
  F1 Score: 0.2016

Poids du modèle local pour Europe:
  Intercept: 0.206163
  Drive